## Physics Teacher v1

Same Gradio interface but now with memory, also replacing gpt-4o-mini with gemini.





In [4]:
# imports
# Import necessary libraries
import os                     # For accessing environment variables
from dotenv import load_dotenv  # For loading API keys from a .env file
import gradio as gr           # For building the web UI
from openai import OpenAI     # OpenAI's official Python client


In [5]:
# ====================
# Load environment variables (e.g., your API key) from a `.env` file
# ====================
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

# Ensure the API key is available
if not OPENAI_API_KEY or not GEMINI_API_KEY:
    raise ValueError("API_KEY not found in your .env file!")


In [6]:
# ====================
# Initialize OpenAI client with the loaded API key
# ====================
client_gpt = OpenAI(api_key=OPENAI_API_KEY)
client_gemini = OpenAI(api_key=GEMINI_API_KEY, base_url=GEMINI_BASE_URL)


In [7]:
# ====================
# Define the system behavior for the chatbot (influences tone and personality)
# ====================
system_message = "You are a helpful, concise assistant for a high-school physics teacher." \
    " You will answer questions about physics and provide explanations." \
    " If the subject of the question is unrelated to phisics, you will tell the user you are a physics assistant and the questions should pertain to physics only" \
    " DO NOT answer anything beyond this if it is not related to physics." 

In [8]:
# Streaming function with model selector and memory
def stream_chat(message: str, history: list, model_choice: str):
    # Prepare full chat history
    messages = [{"role": "system", "content": system_message}]
    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})
    messages.append({"role": "user", "content": message})

    # Select client based on user input
    if "gemini" in model_choice:
        client = client_gemini
    else:
        client = client_gpt

    # Stream from selected model
    stream = client.chat.completions.create(
        model=model_choice,
        messages=messages,
        stream=True
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response



In [43]:
# Build Gradio UI
gr.ChatInterface(
    fn=stream_chat,
    chatbot=gr.Chatbot(label="Multi-Model Chatbot"),
    additional_inputs=[
        gr.Dropdown(
            choices=["gpt-4o-mini", "gemini-2.0-flash"],
            label="Select Model",
            value="gpt-4o-mini"
        )
    ],
    textbox=gr.Textbox(placeholder="Ask anything..."),
    title="🤖 GPT vs Gemini Chatbot",
    theme="default"
).launch()


C:\Users\deluc\AppData\Local\Temp\ipykernel_13120\3789825387.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(label="Multi-Model Chatbot"),
c:\Users\deluc\Documents\Agentic-Code\projects\agents\.venv\Lib\site-packages\gradio\chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.
